### JULIO CESAR GARCIA RIBEIRO
### MATHEUS FELIPE MILESKI LOPES
### FELIPE GALVAO GREGORIO

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
import math, random
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import copy
import os

## Introdução
A agência espacial Europeia European Space Agency - ESA tem um programa chamado GAIA, cuja maior missão é criar o maior e mais preciso mapa tridimensional da nossa galaxia.

A universidade do Canadá, Waterloo utilizou os dados coletados pela ESA para criar 12 instâncias do problema do Caixeiro viajante (Travelling Salesman Problem - TSP) . As instâncias dos problemas vão de 100 até 1,33 bilhão de estrelas possíveis de serem visitadas em um ambiente tridimensional. 

Neste artigo, vamos analisar o problema para 100 estrelas. Para resolução do problema foram desenvolvidos 3 algoritmos diferentes o Busca Gulosa, Algoritmo Genético e GRASP (Greedy Randomized Adaptive Search Procedures). Busca Gulosa é um algoritmo de busca pela melhor escolha que utiliza uma heurística para avaliar sua busca. Algoritmo Genético é um algoritmo inspirados nos modelos de evolução de especies desenvolvidos por Charles Darwin (1859) e por Gregor Mendel (1865), a ideia é simular os conceito da biologia (aptidão, cromossomos, gene, locus, alelos, indivíduos) no algoritmo. GRASP é uma metaheurística constituída por heurísticas construtivas e busca local. Consiste de múltiplas aplicações de busca local, cada uma iniciando de uma solução diferente.


## Objetivos
Esse projeto tem o objetivo de solucionar o problema do caixeiro viajante-TSP aplicando em um exemplo de uma amostra de 100 estrelas.

## Metodologia

Este trabalho se baseou em comparar métodos para encontrar a distância ótima em um problema exponencial do caixeiro viajante. Neste caso, as "cidades" em que o cacheiro precisa passar na realidade são estrelas. Como a amostra possui 100 estrelas o problema então tem um total de 100! possíveis soluções

O dataset utilizado contem as 100 estrelas mais próximas do Sol, e o objetivo é encontrar a rota mais curta para percorrer todas as 100 estrelas passando apenas uma vez por cada uma.

O dataset utilizado neste trabalho pode ser encontrado [aqui](https://www.math.uwaterloo.ca/tsp/star/star100.html). Este dataset foi desenvolvido pela Universidade de Uwaterloo, e contem as coordenadas tridimensionais das 100 estrelas mais próximas do Sol.

Os métodos utilizados para tentar encontrar a rota ótima foram: algoritmo guloso, algoritmo genético e algoritmo GRASP.

### Adquirindo os dados

Para adquirir os dados do dataset, se utilizou a biblioteca Pandas, coletando o arquivo de texto disponível [neste link](https://www.math.uwaterloo.ca/tsp/star/data/star100.xyz.txt). O arquivo de texto possui as coordenadas separadas por espaços, e cada linha representa uma estrela do dataset.

In [ ]:
stars_df = pd.read_csv("https://www.math.uwaterloo.ca/tsp/star/data/star100.xyz.txt", sep = " ", header = None, names = ["x", "y", "z"])
stars_df

,x,y,z
0,0.00000,0.00000,0.00000
1,-4.95181,-4.13973,-11.56674
2,-4.95203,-4.14084,-11.56625
3,-4.72264,-3.61451,-11.51219
4,-0.17373,-18.16613,1.49123
...,...,...,...
95,55.38462,44.88223,8.61013
96,-11.76301,-52.54468,51.42212
97,16.35267,1.84517,-72.75205
98,51.09768,44.51981,32.35931


### Algoritmo guloso

O algoritmo guloso foi desenvolvido para que sempre escolhesse a estrela mais próxima da estrela atual. Saindo do sol, o algoritmo encontra a estrela mais próxima para visitar e remove o Sol do dataset. Este processo é repetido até que as estrelas do dataset se esgotem.

In [ ]:
def calculate_distance(c1, c2):
  return math.sqrt(
    (c2[0] - c1[0])**2 + 
    (c2[1] - c1[1])**2 + 
    (c2[2] - c1[2])**2
  )


def get_nearest_star(current_star, star_df):
  coordinates = star_df.loc[current_star, :].values.tolist()
  star_df.drop(current_star, inplace=True)
  
  distances = []
  for index, star in star_df.iterrows():
    distances.append((index, calculate_distance(coordinates, star)))
  
  return sorted(distances, key=lambda tup: tup[1])[0]


def tsp_greedy(stars_df):
  tuor = [0]
  distance = 0

  next_star = get_nearest_star(tuor[-1], stars_df)
  tuor.append(next_star[0])
  distance += next_star[1]

  while(len(stars_df.index) > 1):
    next_star = get_nearest_star(tuor[-1], stars_df)
    tuor.append(next_star[0])
    distance += next_star[1]
  
  return (distance, tuor)

### Algoritmo genético

O algoritmo genético, a cada iteração (geração), tente a encontrar melhores resultados. Este algoritmo gera rotas aleatórias, sempre saindo do Sol e percorrendo todas as estrelas sem repetições, e gera novas rotas baseadas nas melhores rotas da geração passada.

Para isso, o algoritmo precisa escolher quais rotas serão utilizadas para gerar as novas rotas na nova geração. A função escolhida para isso foi o torneio, onde o algoritmo seleciona n rotas aleatórias da geração atual e escolhe a melhor rota (com menor distância) das rotas selecionadas para o torneio.

Para gerar as novas rotas, os pais escolhidos no torneio são recombinados utilizado o processo de crossover. Neste processo, são gerados dois filhos combinando os genes (estrelas) dos dois pais, sempre mantendo a primeira estrela da rota como sendo o Sol.

Por fim, foi implementado um processo para incluir mutação nas novas rotas. Esse processo utiliza uma taxa de mutação para trocar a posição dos genes. Para cada filho, é seleciona uma posição, partindo da segunda, até chegar ao meio da rota. Se um valor aleatório for superior à taxa de mutação, é realizada a troca de posição pelo elemento espelhado do outro lado da rota.

Os filhos gerados substituem as piores rodas da geração anterior, fazendo com que as melhores rotas da geração atual sejam mantidas na próxima geração.

Existem 5 parâmetros que podem ser alterados para modificar a performance do algoritmo: GENERATIONS: o número de gerações que será considerado (critério de parada do algoritmo); POPULATION_SIZE: tamanho da população que será gerada (quantas rotas em cada geração); TOURNAMENT_SIZE: quantidade de rotas que serão escolhidas aleatoriamente da população para o torneio; NUM_CHILDREN: quantidade de filhos que serão produzidos em cada nova geração; MUTATION_RATE: taxa de mutação (quanto maior mais aleatória serão as rodas filhas de cada geração).

In [ ]:
GENERATIONS = 5000
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 5
NUM_CHILDREN = 40
MUTATION_RATE = 0.05


def create_route(stars_df):
    return [0] + list(random.sample(range(1, len(stars_df.index)), 99))


def initial_population(stars_df):
    population = []
    for i in range(0, POPULATION_SIZE):
        population.append(create_route(stars_df))
    return population


def distance(star1, star2):
    return math.sqrt(
        ((star1[0] - star2[0]) ** 2) + 
        ((star1[1] - star2[1]) ** 2) + 
        ((star1[2] - star2[2])) ** 2
    )


def route_distance(route, stars_df):
    route_distace = 0
    for i in range(0, len(route)-1):
        star1 = stars_df.loc[route[i], :].values.tolist()
        star2 = stars_df.loc[route[i+1], :].values.tolist()
        route_distace += distance(star1, star2)
    return route_distace


def population_fitness(population, stars_df):
    fitness = []
    for route in population:
        distance = route_distance(route, stars_df)
        fitness.append((route, distance, 1/distance))
    return sorted(fitness, key = lambda tup: tup[1])


def crossover(parent_1, parent_2):
    child_1 = [parent_1[0]] + [-1 for i in range(1, len(parent_1))]
    child_2 = [parent_2[0]] + [-1 for i in range(1, len(parent_1))]
    
    delimiters = sorted(random.sample(range(1, len(parent_1)), 2))
    
    for i in range(delimiters[0], delimiters[1]+1):
        child_1[i] = parent_1[i]
        child_2[i] = parent_2[i]

    parent_2_index = 0
    for child_1_index, child_1_cromossome in enumerate(child_1):
        if child_1_cromossome == -1:
            while parent_2[parent_2_index] in child_1:
                parent_2_index += 1
            child_1[child_1_index] = parent_2[parent_2_index]
    
    parent_1_index = 0
    for child_2_index, child_2_cromossome in enumerate(child_2):
        if child_2_cromossome == -1:
            while parent_1[parent_1_index] in child_2:
                parent_1_index += 1
            child_2[child_2_index] = parent_1[parent_1_index]

    return [child_1, child_2]


def breed_population(fitness):
    children = []
    for i in range(0, int(NUM_CHILDREN/2)):
        parents = []
        for j in range(0, 2):
            tournament = random.sample(fitness, TOURNAMENT_SIZE)
            parents.append(sorted(tournament, key = lambda tup: tup[2], reverse = True)[0][0])
        children.extend(crossover(parents[0], parents[1]))
    return children


def mutate(child):
    mutated_child = child
    for i in range(1, len(mutated_child)):
        if random.random() < MUTATION_RATE:
            aux = mutated_child[i]
            mutated_child[i] = mutated_child[len(mutated_child)-1-i]
            mutated_child[len(mutated_child)-1-i] = aux
    return mutated_child


def mutate_population(children):
    mutated_population = []
    for child in children:
        mutated_population.append(mutate(child))
    return mutated_population


def merge_children(fitness, mutated_children):
    new_population = []
    for i in range(0, len(fitness) - len(mutated_children)):
        new_population.append(fitness[i][0])
    return new_population + mutated_children


def next_generation(population, stars_df):
    fitness = population_fitness(population, stars_df)
    best_previous_gen_route = (fitness[0][0], fitness[0][1])
    children = breed_population(fitness)
    mutated_children = mutate_population(children)
    new_generation = merge_children(fitness, mutated_children)
    return (new_generation, best_previous_gen_route)


def genetic_tsp(stars_df):
    best_routes = []
    best_routes_distances = []
    for i in range(0, GENERATIONS+1):
        if i == 0:
            population = initial_population(stars_df)
        else:
            population, best_previous_gen_route = next_generation(population, stars_df)
            best_routes.append(best_previous_gen_route[0])
            best_routes_distances.append(best_previous_gen_route[1])
            # print(f"Generation {i}: Best route distance = {best_routes_distances[-1]}")
    
    plt.plot(best_routes_distances)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

    return (min(best_routes_distances), best_routes[best_routes_distances.index(min(best_routes_distances))])

### Algoritmo GRASP (Greedy Randomized Adaptive Search Procedure)

O GRASP é um algoritmo de otimização combinatória que consiste em criar uma solução inicial aleatória (seed) e depois efetuar uma busca local para melhorar a solução.

In [ ]:
# Function: Tour Distance
def distance_calc(Xdata, star_tour):
    distance = 0
    for k in range(0, len(star_tour[0])-1):
        distance += Xdata[star_tour[0][k]-1, star_tour[0][k+1]-1]
    return distance

# Function: Euclidean Distance 
def euclidean_distance(x, y, z):       
    distance = 0
    for j in range(0, len(x)):
        distance += (x[j] - y[j])**2
    return distance**(1/2) 

# Function: Initial Seed
def seed_function(Xdata):
    seed = [[],float("inf")]
    sequence = []
    sequence = random.sample(list(range(2,Xdata.shape[0]+1)), Xdata.shape[0]-1)
    sequence.insert(0,1)
    sequence.append(1)
    seed[0] = sequence
    seed[1] = distance_calc(Xdata, seed)
    return seed

# Function: Build Distance Matrix
def build_distance_matrix(coordinates):
   a = coordinates
   b = a.reshape(np.prod(a.shape[:-1]), 1, a.shape[-1])
   return np.sqrt(np.einsum('ijk,ijk->ij',  b - a,  b - a)).squeeze()

# Function: Tour Plot
def plot_tour_distance_matrix (Xdata, star_tour):
    m = np.copy(Xdata)
    for i in range(0, Xdata.shape[0]):
        for j in range(0, Xdata.shape[1]):
            m[i,j] = (1/2)*(Xdata[0,j]**2 + Xdata[i,0]**2 - Xdata[i,j]**2)    
    w, u = np.linalg.eig(np.matmul(m.T, m))
    s = (np.diag(np.sort(w)[::-1]))**(1/2) 
    coordinates = np.matmul(u, s**(1/2))
    coordinates = coordinates.real[:,0:2]
    xyz = np.zeros((len(star_tour[0]), 3))
    print(xyz)
    for i in range(0, len(star_tour[0])):
        if (i < len(star_tour[0])):
            xyz[i, 0] = coordinates[star_tour[0][i]-1, 0]
            xyz[i, 1] = coordinates[star_tour[0][i]-1, 1]
            xyz[i, 2] = coordinates[star_tour[0][i]-1, 2]
        else:
            xyz[i, 0] = coordinates[star_tour[0][0]-1, 0]
            xyz[i, 1] = coordinates[star_tour[0][0]-1, 1]
            xyz[i, 2] = coordinates[star_tour[0][0]-1, 2]
    plt.plot(xyz[:,0], xyz[:,1], xyz[:,2], marker = 's', alpha = 1, markersize = 7, color = 'black')
    plt.plot(xyz[0,0], xyz[0,1], xyz[0,2], marker = 's', alpha = 1, markersize = 7, color = 'red')
    plt.plot(xyz[1,0], xyz[1,1], xyz[1,2], marker = 's', alpha = 1, markersize = 7, color = 'orange')
    return

# Function: Tour Plot
def plot_tour_coordinates (coordinates, star_tour):
    xyz = np.zeros((len(star_tour[0]), 3))
    for i in range(0, len(star_tour[0])):
        if (i < len(star_tour[0])):
            xyz[i, 0] = coordinates[star_tour[0][i]-1, 0]
            xyz[i, 1] = coordinates[star_tour[0][i]-1, 1]
            xyz[i, 2] = coordinates[star_tour[0][i]-1, 2]
        else:
            xyz[i, 0] = coordinates[star_tour[0][0]-1, 0]
            xyz[i, 1] = coordinates[star_tour[0][0]-1, 1]
            xyz[i, 2] = coordinates[star_tour[0][0]-1, 2]
    plt.plot(xyz[:,0], xyz[:,1], xyz[:,2], marker = 's', alpha = 1, markersize = 7, color = 'black')
    plt.plot(xyz[0,0], xyz[0,1], xyz[0,2], marker = 's', alpha = 1, markersize = 7, color = 'red')
    plt.plot(xyz[1,0], xyz[1,1], xyz[1,2], marker = 's', alpha = 1, markersize = 7, color = 'orange')
    plt.show()
    return

# Function: Rank Cities by Distance
def ranking(Xdata, star = 0):
    rank = np.zeros((Xdata.shape[0], 2)) # ['Distance', 'star']
    for i in range(0, rank.shape[0]):
        rank[i,0] = Xdata[i,star]
        rank[i,1] = i + 1
    rank = rank[rank[:,0].argsort()]
    return rank

# Function: RCL
def restricted_candidate_list(Xdata, greediness_value = 0.5):
    seed = [[],float("inf")]
    sequence = random.sample(list(range(2,Xdata.shape[0]+1)), Xdata.shape[0]-1)
    sequence.insert(0,1)
    count = 1
    for i in range(1, Xdata.shape[0]):
        count = 1
        rand = int.from_bytes(os.urandom(8), byteorder = "big") / ((1 << 64) - 1)
        if (rand > greediness_value and len(sequence) < Xdata.shape[0]):
            next_star = int(ranking(Xdata, star = sequence[-1] - 1)[count,1])
            while next_star in sequence:
                count = np.clip(count+1,1,Xdata.shape[0]-1)
                next_star = int(ranking(Xdata, star = sequence[-1] - 1)[count,1])
            sequence.append(next_star)
        elif (rand <= greediness_value and len(sequence) < Xdata.shape[0]):
            next_star = random.sample(list(range(1,Xdata.shape[0]+1)), 1)[0]
            while next_star in sequence:
                next_star = int(random.sample(list(range(1,Xdata.shape[0]+1)), 1)[0])
            sequence.append(next_star)
    sequence.append(1)
    seed[0] = sequence
    seed[1] = distance_calc(Xdata, seed)
    return seed

# Function: 2_opt
def local_search_2_opt(Xdata, star_tour):
    tour = copy.deepcopy(star_tour)
    best_route = copy.deepcopy(tour)
    seed = copy.deepcopy(tour)  
    for i in range(0, len(tour[0]) - 2):
        for j in range(i+1, len(tour[0]) - 1):
            best_route[0][i:j+1] = list(reversed(best_route[0][i:j+1]))           
            best_route[0][-1]  = best_route[0][0]                          
            best_route[1] = distance_calc(Xdata, best_route)           
            if (best_route[1] < tour[1]):
                tour[1] = copy.deepcopy(best_route[1])
                for n in range(0, len(tour[0])): 
                    tour[0][n] = best_route[0][n]          
            best_route = copy.deepcopy(seed) 
    return tour

def greedy_randomized_adaptive_search_procedure(Xdata, star_tour, iterations = 50, rcl = 25, greediness_value = 0.5):
    count = 0
    best_solution = copy.deepcopy(star_tour)
    while (count < iterations):
        rcl_list = []
        for i in range(0, rcl):
            rcl_list.append(restricted_candidate_list(Xdata, greediness_value = greediness_value))
        candidate = int(random.sample(list(range(0,rcl)), 1)[0])
        star_tour = local_search_2_opt(Xdata, star_tour = rcl_list[candidate])
        while (star_tour[0] != rcl_list[candidate][0]):
            rcl_list[candidate] = copy.deepcopy(star_tour)
            star_tour = local_search_2_opt(Xdata, star_tour = rcl_list[candidate])
        if (star_tour[1] < best_solution[1]):
            best_solution = copy.deepcopy(star_tour) 
        count = count + 1
        print("Iteration =", count, "-> Distance =", best_solution[1])
    print("Best Solution =", best_solution)
    return best_solution

## Resultados

### Algoritmo guloso

Este algoritmo produz apenas uma saída. A rota encontrada possui distancia total de 2056,20.

In [ ]:
greedy_df = stars_df.copy()
distance, tuor = tsp_greedy(greedy_df)
print(f"Distância: {distance}")
print(f"Caminho: {tuor}")

Distância: 2056.203289334893
Caminho: [0, 3, 1, 2, 4, 7, 23, 41, 16, 9, 28, 40, 38, 35, 66, 75, 91, 86, 46, 64, 44, 77, 82, 69, 63, 55, 45, 31, 27, 20, 59, 74, 76, 68, 56, 12, 11, 24, 17, 43, 84, 70, 52, 51, 81, 60, 78, 39, 37, 5, 10, 34, 48, 50, 85, 79, 80, 65, 62, 36, 25, 29, 67, 89, 33, 53, 49, 94, 92, 88, 15, 21, 26, 6, 22, 8, 19, 18, 30, 99, 95, 98, 42, 54, 57, 72, 61, 73, 87, 97, 13, 14, 32, 58, 93, 83, 71, 96, 90, 47]


### Algoritmo genético

O resultado desse algoritmo depende muito dos parâmetros fornecidos. Foram realizados 4 testes utilizando esse algoritmo, e os gráficos de distância por geração podem ser observados [neste arquivo](https://docs.google.com/document/d/1fs465ihFLieTey5VyBW7GScAJwXHda-ojaokzPuqXCw/edit?usp=sharing).

Os resultados desse algoritmo não foram superiores ao algoritmo guloso. O algoritmo acaba travando em soluções ótimas locais, e o melhor resultado gerou uma rota com distância de cerca de 3500.

Este resultado pode ser aprimorado aumentado a aleatoriedade dos filhos (aumentando a taxa de mutação), aumentando o número de gerações ou utilizando outros métodos juntamente ao algoritmo genético, como o 2-opt (método de procura local para encontrar rotas com menores distâncias) em certas gerações.


In [ ]:
genetic_df = stars_df.copy()
distance, tuor = genetic_tsp(stars_df = genetic_df)
print(f"Menor distância encontrada: {distance}")
print(f"Melhor caminho encontrada: {tuor}")

NameError: ignored

### Algoritmo GRASP (Greedy Randomized Adaptive Search Procedure)

O resultado do GRASP estabilizou por volta de 10 iterações, com uma distância de 1824.


In [ ]:
grasp_df = stars_df.copy()
Y = grasp_df.values

# Build the Distance Matrix
X = build_distance_matrix(Y)

# Start a Random Seed
seed = seed_function(X)

# Call the Function
lsgrasp = greedy_randomized_adaptive_search_procedure(X, star_tour = seed, iterations=15)

# Plot Solution. Red Point = Initial star; Orange Point = Second star
plot_tour_coordinates(Y, lsgrasp)

Iteration = 1 -> Distance = 1957.3367688992857
Iteration = 2 -> Distance = 1921.3704037496927
Iteration = 3 -> Distance = 1886.2326830722711
Iteration = 4 -> Distance = 1877.7490687116715
Iteration = 5 -> Distance = 1877.7490687116715
Iteration = 6 -> Distance = 1877.7490687116715


KeyboardInterrupt: ignored